In [1]:
import os
from scripts.paper_figures.input import Input

import basty.project.experiment_processing as experiment_processing
from scripts.paper_figures.process_results import BehaviorData

from basty.utils import misc

In [2]:

RESULTS_FOLDER = r'Z:\mfk\basty-projects\backup-allHaltere\results\semisupervised_pair_kNN\predictions.100NN.neighbor_weights-distance.neighbor_weights_norm-count.activation-standard.voting-soft.voting_weights-None\exports'

BODY_PART_SETS = {"HaltereSwitch": "prob", "Feeding": "prob"}

CONFIG_PATH = r"Z:\mfk\basty-projects\main_cfg.yaml"

PROJECT_PATH = os.path.dirname(CONFIG_PATH)

TMP_RESULT_FOLDER = os.path.join(PROJECT_PATH,'tmp_results')

project = experiment_processing.Project(CONFIG_PATH)

BEHAVIORS = ['HaltereSwitch']

BODY_PART_SETS = {"ProboscisPumping": "prob", "Feeding": "prob", "HaltereSwitch": "halt"}

In [3]:
io_process = Input(project, RESULTS_FOLDER, TMP_RESULT_FOLDER)

In [4]:
expt_info_df = io_process.load_expt_info()
df_data = io_process.load_predictions()

In [5]:
# Drop Noise, Feeding, Grooming, ProboscisPumping from df_data
df_data.drop(['Noise','Feeding', 'Grooming', 'ProboscisPumping'], axis=1, inplace=True)

io_process.create_binary_masks_subfolders(BEHAVIORS)

In [6]:
llh = misc.get_likelihood(project.data_path_dict, CONFIG_PATH)

Loading existing llh_df.pickle file...


In [7]:
llh = llh[llh["ExptNames"].isin(df_data.ExptNames.unique())]

In [8]:
from importlib import reload
import scripts.paper_figures.process_results
reload(scripts.paper_figures.process_results)
from scripts.paper_figures.process_results import BehaviorData

process_data = BehaviorData(df_data, binary_mask_threshold=0.7,body_parts=BODY_PART_SETS['HaltereSwitch'])

process_data.process_expt_names_parallel(llh,io_process.get_binary_mask_subfolder(behavior='HaltereSwitch'))

File Fly05182022_5d_median6_threshold0.7.pkl already exists. Skipping calculation.File Fly05202022_5d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly05222022_5d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly05252022_SD_5d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly05272022_SD_5d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly06072022_5d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly06082022_5d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly06152022_6d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly06142022_SD_5d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly06182022_4d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly06212022_5d_median6_threshold0.7.pkl already exists. Skipping calculation.
File Fly06202022_5d_median6_threshold0.7.pkl already exists. Skipping calculati

In [16]:
masks_based_on_llh = io_process.load_binary_mask('HaltereSwitch')
beh_masks = process_data.create_binary_mask_from_behaviors(BEHAVIORS,'HaltereSwitch')
masked_data = process_data.update_dictionary_with_final_masked(masks_based_on_llh,beh_masks)

In [22]:
# Remove all the haltere switch events that scored less than 0.5

# Masked data is a dictionary, so for each key, we have a dataframe get the column name that ends with _final_masked

for key in masked_data.keys():
    # Get the DataFrame associated with the current key
    df = masked_data[key]
    
    # Filter the column(s) of interest
    column_filtered = df.filter(regex='_final_masked$').columns[0]
    
    # Apply the transformation to the filtered column
    df[column_filtered] = df[column_filtered].apply(lambda x: x if x > 0.5 else 0)
    
    # Update the modified DataFrame back into the dictionary
    masked_data[key] = df



In [25]:
bouts_dict = BehaviorData.find_consecutive_bouts_and_snap_fts(masked_data, io_process, 'HaltereSwitch', 6, 30*5, 60, True)

# For each unique key in the bouts dict, process each row such that we grab a specific column called 'distance.thor_post-halt'


Processing Experiments: 100%|██████████| 61/61 [01:18<00:00,  1.29s/it] 


In [43]:
import plotter
ploting = plotter.Plotter()

ploting.plot_time_series(bouts_dict,'distance.thor_post-halt','multipage_haltere_switch.pdf')

In [42]:
bouts_dict['Fly05182022_5d']

,start_index,stop_index,region,pose.prob_x,pose.prob_y,pose.halt_x,pose.halt_y,pose.thor_post_x,pose.thor_post_y,distance.origin-halt,distance.origin-prob,distance.origin-thor_post,distance.head-prob,distance.thor_post-halt,"distance.avg(thor_post-joint1,thor_post-joint2,thor_post-joint3)","distance.avg(origin-joint1,origin-joint2,origin-joint3)"
0,12627,13420,Fly05182022_5d,"[761.4926316142082, 761.1564144690832, 760.903...","[646.007684459289, 645.9407633443674, 646.0039...","[594.3264920711517, 593.8677310943604, 593.367...","[590.4707633256912, 590.5348969697952, 590.684...","[600.272825901707, 599.892067377766, 599.51130...","[551.2048719525337, 551.0998704632124, 551.047...","[837.7826099412814, 837.5024457124231, 837.253...","[998.5974946810073, 998.2978298270185, 998.145...","[814.9566101211955, 814.6051557210644, 814.289...","[84.52971487202653, 84.09112192541473, 83.7965...","[39.713588497553474, 39.89252991764812, 40.110...","[144.52681980674265, 144.64184586538912, 144.7...","[916.9363904018384, 916.8318919525997, 916.727..."
1,20674,21013,Fly05182022_5d,"[760.7512847129256, 761.073934243992, 760.6809...","[638.5907069494327, 638.8241302023331, 639.057...","[589.5968303481737, 589.748064160347, 589.8947...","[588.9444858233134, 589.1253223816553, 589.329...","[597.4055593808492, 597.3121384779612, 597.218...","[552.2773360411326, 552.3034802079201, 552.372...","[833.3546842361301, 833.5894820906548, 833.837...","[993.2475060097272, 993.6447064793309, 993.493...","[813.5746175267733, 813.5237704104874, 813.502...","[81.77012933219075, 82.08837417668498, 82.0704...","[37.48941346906657, 37.59073398259017, 37.6758...","[141.71240990894202, 141.65717395302704, 141.6...","[917.973926279869, 917.9316166615417, 917.9814..."
2,23499,23895,Fly05182022_5d,"[757.7259101048112, 757.6293196777502, 757.532...","[637.3598196518918, 637.2034639542302, 637.351...","[589.4289827942848, 589.453923424085, 589.3493...","[589.5680146813393, 589.5454498728117, 589.574...","[598.3388629953066, 598.5969821214676, 598.329...","[552.5107916792234, 552.5102991461754, 552.542...","[833.6767777101634, 833.6784543850123, 833.625...","[990.1394318735399, 989.964868321417, 989.9863...","[814.4185471196727, 814.6078674230471, 814.432...","[79.74871446745404, 79.3478222645908, 79.24704...","[38.11330137662053, 38.14705639611596, 38.1056...","[141.9028222085635, 141.93806724943383, 142.08...","[918.3133429542609, 918.4950339877154, 918.625..."
3,46189,46522,Fly05182022_5d,"[759.6255396207174, 759.5564039548238, 759.560...","[642.0345052282015, 642.2221019690236, 642.468...","[590.410032749176, 590.1877482136091, 590.0478...","[588.9892371408641, 589.1649510071924, 589.514...","[598.4817476669947, 598.3653322060903, 598.248...","[552.5429235597452, 552.7321875592073, 553.021...","[833.9618266075853, 833.9286046399643, 834.076...","[994.6050805961568, 994.6733931528954, 994.835...","[814.5453238871921, 814.5882039104738, 814.699...","[84.6331220556486, 84.71846241128465, 84.88725...","[37.329430150590184, 37.33924386501314, 37.402...","[140.8466486751838, 140.8525163971865, 140.837...","[916.1350666920756, 916.1568049078397, 916.242..."
4,66435,66791,Fly05182022_5d,"[760.48686170578, 760.650934179624, 760.900248...","[639.0799893538157, 639.3919371962547, 639.703...","[594.0198311209679, 594.049069583416, 593.9873...","[587.2529713461796, 587.3636640831828, 587.474...","[599.7418003777663, 599.6932088335356, 599.644...","[552.2510998447736, 552.2259312272072, 552.235...","[835.2997139469761, 835.3983307130463, 835.432...","[993.3597030379174, 993.6860132959073, 994.077...","[815.273883060243, 815.2210889328236, 815.1918...","[80.6006032016345, 80.7927607402326, 81.019970...","[35.466490392716906, 35.58815221576767, 35.690...","[140.14127926377606, 140.33489407369458, 140.5...","[916.6359275888727, 916.8006324657017, 916.958..."
5,70249,70658,Fly05182022_5d,"[760.6731842756271, 760.6889368494352, 760.752...","[638.4161148096124, 638.5362859976789, 638.6

In [15]:
expt_name = 'Fly05182022_5d'

sub_likelihood_data  = llh[llh['ExptNames']==expt_name]
sub_behavior_data = process_data.data[process_data.data['ExptNames']==expt_name]


In [18]:
sub_likelihood_data['halt_l'] = sub_likelihood_data[['halt_l', 'halt_r']].max(axis=1)

In [19]:
sub_likelihood_data.halt_l

0          0.945897
1          0.954843
2          0.969784
3          0.957228
4          0.884038
             ...   
1727995    0.027633
1727996    0.619227
1727997    0.889487
1727998    0.936911
1727999    0.026978
Name: halt_l, Length: 1728000, dtype: float64

In [ ]:



#TODO: PASS FIG PATH TO DOWNSTREAM
class Args:
    def __init__(self):
        self.CONFIG_PATH = r"Z:\mfk\basty-projects\main_cfg.yaml"
        self.PROJECT_PATH = os.path.dirname(self.CONFIG_PATH)
        self._get_tmp_result_folder()
        self.FPS = 30
        self.FIG_PATH = r"C:\Users\Grover\Documents\GitHub\deepsleepfigs"
        self.RESULTS_FOLDER = r"Z:\mfk\basty-projects\backup-allnohaltere\results\semisupervised_pair_kNN\predictions.15NN.neighbor_weights-distance.neighbor_weights_norm-log_count.activation-standard.voting-soft.voting_weights-None\exports"
        self.BODY_PART_SETS = {"ProboscisPumping": "prob", "Feeding": "prob"}
        self.BEHAVIORS = [
            "Idle&Other",
            "PosturalAdjustment&Moving",
            "Feeding",
            "Grooming",
            "ProboscisPumping",
        ]
        self.output_path = r'Z:\mfk\basty-projects'
        self._get_tmp_result_folder()

    def _get_tmp_result_folder(self):
        tmp_result_folder = os.path.join(self.PROJECT_PATH,'tmp_results')

        if not os.path.exists(tmp_result_folder):
            os.makedirs(tmp_result_folder)

        self.TMP_RESULT_FOLDER = tmp_result_folder
